In [76]:
import os, glob, platform
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.utils.data as data_utils
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.autograd import Variable

import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

import cv2
from PIL import Image

import numpy as np
from numpy.linalg import inv as denseinv
from scipy import sparse
from scipy.sparse import lil_matrix, csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import inv as spinv
import scipy.misc

# Configurations

In [77]:
class Args(object):
    pass
args = Args()
args.test_scene = 'alley_1'
args.arch = "densenet121"
args.epoches = 500
args.epoches_unary_threshold = 0
args.image_h = 256
args.image_w = 256
args.img_extentions = ["png"]
args.training_thresholds = [250,200,150,50,0,300]
args.base_lr = 1e4
args.snapshot_interval = 5000
args.debug = True
args.gpu_num = 1
args.display_interval = 50
args.display_curindex = 0

system_ = platform.system()
system_dist, system_version, _ = platform.dist()
if system_ == "Darwin": 
    args.train_dir = '/Volumes/Transcend/dataset/sintel2'
    args.pretrained = False
elif platform.dist() ==  ('debian', 'jessie/sid', ''):
    args.train_dir = '/home/albertxavier/dataset/sintel2'
    args.pretrained = True
elif platform.dist() == ('debian', 'stretch/sid', ''):
    args.train_dir = '/home/cad/lwp/workspace/dataset/sintel2'
    args.pretrained = True
print(platform.dist())

('debian', 'stretch/sid', '')


# Little Test

In [78]:
# class Netxxx(nn.Module):
#     def __init__(self):
#         super(Netxxx, self).__init__()
#         self.bn = nn.BatchNorm2d(64)
#     def forward(self,x):
#         return self.bn(x)
# netxxx = Netxxx()
# netxxx = netxxx.cuda(args.gpu_num)
# x = Variable(torch.rand(1,64,5,6)).cuda(args.gpu_num)
# y = netxxx(x)


# My DataLoader

In [79]:
def default_loader(path):
    return Image.open(path).convert('RGB')

def make_dataset(dir, phase):
    images_paths = glob.glob(os.path.join(dir, 'clean', '*', '*.png'))
    albedo_paths = images_paths[:]
    shading_paths = images_paths[:]
    pathes = []
    for img_path in images_paths:
        sp = img_path.split('/'); 
        if phase == 'train':
            if sp[-2] == args.test_scene: continue
        else:
            if sp[-2] != args.test_scene: continue
            
        sp[-3] = 'albedo'; 
        sp = ['/'] + sp; 
        albedo_path = os.path.join(*sp)
        
        sp = img_path.split('/'); 
        sp[-3] = 'shading'; 
        sp[-1] = sp[-1].replace('frame', 'out')
        sp = ['/'] + sp; 
        shading_path = os.path.join(*sp)
        
        pathes.append((img_path, albedo_path, shading_path))
    return pathes

class MyImageFolder(data_utils.Dataset):
    def __init__(self, root, phase='train', transform=None, target_transform=None,
                loader=default_loader):
        imgs = make_dataset(root, phase)
        if len(imgs) == 0:
            raise(RuntimeError("Found 0 images in subfolders of: " + root + "\n"
                               "Supported image extensions are: " + ",".join(args.img_extentions)))

        self.root = root
        self.imgs = imgs
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader
        
    def __getitem__(self, index):
        img_path, albedo_path, shading_path = self.imgs[index]
        
        img = self.loader(img_path)
        albedo = self.loader(albedo_path)
        shading = self.loader(shading_path)
        
        if self.transform is not None: img = self.transform(img)
        if self.transform is not None: albedo = self.transform(albedo)
        if self.transform is not None: shading = self.transform(shading)
        
        scene = img_path.split('/')[-2]
        return img, albedo, shading, scene, img_path
    
    def __len__(self):
        return len(self.imgs)
    
train_dataset = MyImageFolder(args.train_dir, 'train', 
                       transforms.Compose(
        [transforms.RandomCrop((args.image_h, args.image_w)),
         transforms.ToTensor()]
    ))
test_dataset = MyImageFolder(args.train_dir, 'test', 
                       transforms.Compose(
        [transforms.CenterCrop((args.image_h, args.image_w)),
         transforms.ToTensor()]
    ))

train_loader = data_utils.DataLoader(train_dataset,1,True,num_workers=1)
test_loader = data_utils.DataLoader(test_dataset,1,False,num_workers=1)

# Load Pretrained Model

[Defination](https://github.com/pytorch/vision/blob/master/torchvision/models/densenet.py)
* DenseNet-121: num_init_features=64, growth_rate=32, block_config=(6, 12, 24, 16)
    * First Convolution: 32M -> 16M -> 8M
    * every transition: 8M -> 4M -> 2M (downsample 1/2, except the last block)

In [80]:
densenet = models.__dict__[args.arch](pretrained=args.pretrained)

# for param in densenet.parameters():
#     param.requires_grad = False
if platform.system() == 'Linux':
    densenet = densenet.cuda(args.gpu_num)


In [81]:
class PreTrainedModel(nn.Module):
    def __init__(self, pretrained):
        super(PreTrainedModel, self).__init__()
        common_features_net = nn.Sequential(*list(pretrained.children())[0:1])
        self.net_16M = nn.Sequential(OrderedDict([
            ('conv0', common_features_net[0].conv0),
            ('norm0', common_features_net[0].norm0),
            ('relu0', common_features_net[0].relu0)
        ]))
        self.net_8M = nn.Sequential(OrderedDict([
            ('pool0', common_features_net[0].pool0)
        ]))
        self.net_4M = nn.Sequential(OrderedDict([
            ('denseblock1', common_features_net[0].denseblock1),
            ('transition1', common_features_net[0].transition1)
        ]))
        self.net_2M = nn.Sequential(OrderedDict([
            ('denseblock2', common_features_net[0].denseblock2),
            ('transition2', common_features_net[0].transition2)
        ]))
        self.net_1M = nn.Sequential(OrderedDict([
            ('denseblock3', common_features_net[0].denseblock3),
            ('transition3', common_features_net[0].transition3),
            ('denseblock4', common_features_net[0].denseblock4)
        ]))
    def forward(self, ft_32M):
        
        pretrained_features = [0]*5
        pretrained_features[0] = self.net_16M(ft_32M)
        pretrained_features[1]  = self.net_8M(pretrained_features[0])
        pretrained_features[2]  = self.net_4M(pretrained_features[1])
        pretrained_features[3]  = self.net_2M(pretrained_features[2])
        pretrained_features[4]  = self.net_1M(pretrained_features[3])
        return pretrained_features.cuda(args.gpu_num)

if args.debug == True:
#     pass
    print(densenet)
#     common_features_net = nn.Sequential(*list(densenet.children())[0:1])
#     net_x = nn.Sequential(OrderedDict([
#             ('conv0', common_features_net[0].conv0),
#     ]))
    
#     """
#         debug: copy/clone
# #     """
# #     print net_x
#     t = nn.Sequential(*list(net_x.children()))

#     for param in _8M.parameters():
#         print param.data

#     for param in t.parameters():
#         param.data = (param.data*2)
#     print "@@@@@@"

#     for param in t.parameters():
#         print param.data


DenseNet (
  (features): Sequential (
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (relu0): ReLU (inplace)
    (pool0): MaxPool2d (size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    (denseblock1): _DenseBlock (
      (denselayer1): _DenseLayer (
        (norm.1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU (inplace)
        (conv.1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm.2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (relu.2): ReLU (inplace)
        (conv.2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer (
        (norm.1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU (inplace)
        (conv.1): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [82]:

class _DenseLayer(nn.Sequential):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate):
        super(_DenseLayer, self).__init__()
        self.add_module('norm.1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu.1', nn.ReLU(inplace=True)),
        self.add_module('conv.1', nn.Conv2d(num_input_features, bn_size *
                        growth_rate, kernel_size=1, stride=1, bias=False)),
        self.add_module('norm.2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu.2', nn.ReLU(inplace=True)),
        self.add_module('conv.2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                        kernel_size=3, stride=1, padding=1, bias=False)),
        self.drop_rate = drop_rate

    def forward(self, x):
        new_features = super(_DenseLayer, self).forward(x)
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate, training=self.training)
        return torch.cat([x, new_features], 1)


class _DenseBlock(nn.Sequential):
    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(num_input_features + i * growth_rate, growth_rate, bn_size, drop_rate)
            self.add_module('denselayer%d' % (i + 1), layer)

class _MyTransition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_MyTransition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features, num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=3, stride=1, padding=1))


    
class GradientNet(nn.Module):
    def build_blocks(self, num_block, num_init_features):
        bn_size = 4
        growth_rate = 32
        drop_rate = 0
        num_features = num_init_features
        features = nn.Sequential()
        for i, num_layers in enumerate(num_block):
            block = _DenseBlock(num_layers=num_layers, num_input_features=num_features,
                                bn_size=bn_size, growth_rate=growth_rate, drop_rate=drop_rate)
            features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate
            
            trans = _MyTransition(num_input_features=num_features, num_output_features=num_features // 2)
            features.add_module('transition%d' % (i + 1), trans)
            num_features = num_features // 2
        return features.cuda(args.gpu_num)
    
    def __init__(self, pretrained_model):
        super(GradientNet, self).__init__()
        self.block_config = [(3,3,3),(6,6,6),(12,12,12),(16,16,16),(24,24,24)]
        self.num_input_features = [64,64,128,256,1024]
        self.upsample_config = [2,4,8,16,32]
        
        self.pretrained_model = pretrained_model
        
        self.denseblocks = [nn.Sequential()] * len(self.block_config)
        for i in range(0, len(self.block_config)):
            self.denseblocks[i] = self.build_blocks(self.block_config[i], self.num_input_features[i])
#         self.tmplayer = nn.BatchNorm2d(64)
        
        self.upsamples = [0] * len(self.block_config)
        stride = 2
        self.num_upsample_input_features = [92,176,352,480,800]
        for i in range(0, len(self.block_config)):
            self.upsamples[i] = nn.ConvTranspose2d(in_channels=self.num_upsample_input_features[i], 
                                                   out_channels=3, kernel_size=self.upsample_config[i], 
                                    stride=stride, padding=0, 
                                                   output_padding=0, groups=1, bias=True, dilation=1)
            if platform.system() == 'Linux': 
                self.upsamples[i] = self.upsamples[i].cuda(args.gpu_num)
            stride = stride * 2
        
        self.merge = nn.Sequential()
        self.merge_in_channels =  (3*len(self.block_config), 64, 32, 16)
        self.merge_out_channels = (                      64, 32, 16,  3)
        for i in range(0, len(self.merge_out_channels)): 
            self.merge.add_module('merge.conv{}'.format(i), 
                                  nn.Conv2d(in_channels=self.merge_in_channels[i], 
                                            out_channels=self.merge_out_channels[i], kernel_size=1))
            self.merge.add_module('merge.relu{}'.format(i), nn.ReLU(inplace=True))
                                    
    def forward(self, ft_input):
        ft_pretrained = self.pretrained_model(ft_input)
#         if args.debug == True: 
#             for i,v in enumerate(ft_pretrained): 
#                 print '{}th ft_pretrained shape = {}'.format(i, v.data.shape)
#         print("ft_pretrained[0] = ", ft_pretrained[0])
#         print("denseblocks[0] = ", self.denseblocks[0])
#         tmp1= self.tmplayer(ft_pretrained[0])
        ft_predict = [b(ft_pretrained[i]) for i, b in enumerate(self.denseblocks)]
#         if args.debug == True: 
#             for i,v in enumerate(ft_predict): 
#                 print '{}th ft_predict shape = {}'.format(i, v.data.shape)
        ft_upsampled = [up(ft_predict[i]) for i, up in enumerate(self.upsamples)]
        ft_concated = torch.cat(ft_upsampled, 1)
        ft_merged = self.merge(ft_concated)
        ft_output = ft_upsampled + [ft_merged]
        for k,v in enumerate(ft_output): ft_output[k] = v.cuda(args.gpu_num)
        return ft_output


# Optimizer & Loss Function

In [83]:
pretrained = PreTrainedModel(densenet)
net = GradientNet(pretrained)

if platform.system() == "Linux": 
    net = net.cuda(args.gpu_num)
mse_losses = [nn.MSELoss()] * 6
optimizer = optim.SGD(net.parameters(), lr=args.base_lr, momentum=0.9)

# Training Loop

In [84]:
args.display_curindex = 0
for epoch in range(args.epoches):
    print('epoch: {}'.format(epoch))
    run_losses = [0] * len(args.training_thresholds)
    for i, data in enumerate(train_loader, 0):
        input_img, gt_albedo, gt_shading, test_scene, img_path = data
        if test_scene[0] == 'alley_1':
            print('alley_1 yes')
        input_img = Variable(input_img)
        gt_albedo = Variable(gt_albedo)
        gt_shading = Variable(gt_shading)
        if platform.system() == 'Linux':
            input_img = input_img.cuda(args.gpu_num)
            gt_albedo = gt_albedo.cuda(args.gpu_num)
            gt_shading = gt_shading.cuda(args.gpu_num)
        
        run_losses = [Variable(torch.Tensor([0])).cuda(args.gpu_num)] * len(mse_losses)
        for (i,threshold) in enumerate(args.training_thresholds):
            if epoch >= threshold:
                optimizer.zero_grad()
                ft_predict = net(input_img)
#             if epoch >= 0:
                if i == 5: s = 1.
                else: s = (2**(i+1))
                gt = gt_albedo.cpu().data.numpy()
                n,c,h,w = gt.shape
                gt = gt[0,:,:,:]
                gt = gt.transpose((1,2,0))
                _, _, h2, w2 = ft_predict[i].cpu().data.numpy().shape
                gt = cv2.resize(gt, (h2,w2))
                gt = cv2.resize(gt, (h,w))
                gt = gt.transpose((2,0,1))
                gt = gt[np.newaxis, :]
                gt = Variable(torch.from_numpy(gt))
                if platform.system() == "Linux": gt = gt.cuda(args.gpu_num)
                run_losses[i] = mse_losses[i](ft_predict[i], gt)
                run_losses[i].backward()
                optimizer.step()
                
        
        if args.display_curindex % args.display_interval == 0:
            loss_output = 'ind: ' + str(args.display_curindex)
            for i,v in enumerate(run_losses):
                if i == len(run_losses)-1: 
                    loss_output += ' merged: %6f' % (run_losses[-1].data.cpu().numpy()[0])
                    continue
                loss_output += ' %2dM: %6f' % ((2**(4-i)), (run_losses[i].cpu().data.numpy()[0]))
            print(loss_output)
        args.display_curindex += 1
        
    # save at every epoch
    torch.save({
        'epoch': epoch,
        'args' : args,
        'state_dict': net.state_dict(),
        'optimizer': optimizer.state_dict()
    }, 'snapshot/snapshot-{}.pth.tar'.format(epoch))
    
    # test 
    for i, data in enumerate(test_loader, 0):
        input_img, gt_albedo, gt_shading, test_scene, img_path = data
        input_img = Variable(input_img)
        gt_albedo = Variable(gt_albedo)
        gt_shading = Variable(gt_shading)
        if platform.system() == 'Linux':
            input_img = input_img.cuda(args.gpu_num)
        ft_test = net(input_img)
        for k,v in enumerate(ft_test):
            v = v[0].cpu().data.numpy()
            v = v.transpose(1,2,0)
            print(v.shape, v.min(), v.max())
            cv2.imwrite('snapshot/snapshot-{}-{}.png'.format(epoch, k), v*255)
        break

epoch: 0


AttributeError: 'list' object has no attribute 'cuda'

In [ ]:
# Visualize Graph

In [4]:
from graphviz import Digraph
import torch
from torch.autograd import Variable


def make_dot(var, params=None):
    """ Produces Graphviz representation of PyTorch autograd graph

    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function

    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return '('+(', ').join(['%d' % v for v in size])+')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                name = param_map[id(u)] if params is not None else ''
                node_name = '%s\n %s' % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.creator)
    return dot

ImportError: No module named graphviz

In [ ]:
make